# Guide to Gates Provided by Pyquil

A reference guide for all of the gates that come standard with the Pyquil library, specifically the pyquil.gates file, along with some working examples

-------------------------------------

### Programs and Measurements

A neccessary compoent tot writing code with Pyquil is understanding how a 'quantum circuit' is represented in their python code.  Specifically, Pyquil contains all information about gate sequencing in "Programs".  These programs are best thought of as a set of gate instructions, stored in a single list called a Program.  For example, the code below creates a program that contains the set of instructions: "First, apply a Hadamard Gate to qubit 0, then apply an X gate to qubit 0"

In [3]:
from pyquil import Program
from pyquil.gates import H,X

program = Program( H(0), X(0) )
print(program)

H 0
X 0



As we can see, printing program1 to console shows us the gates we requested, in the order we specified.  This is a handy feature that we will make use of frequently.

So a program is our 'quantum circuit', next we need to observe it.  This can be done one of several ways:

1) **wavefunction()** - This allows us to peak at the wavefunction without making ameasurement

2) **measure()** and **run()** - The standard way of observing a quantum system. 

Let's see examples of both:

In [145]:
from pyquil import QVMConnection
qvm = QVMConnection()

program1 = Program(H(0),H(1))
program2 = Program(H(0),H(1))
wf = qvm.wavefunction(program1)
print('_____The Wavefunction_____')
print(wf)
print('  ')

program1.measure(0,0)
run1 = qvm.run(program1,0)
print('___Measurement on qubit 0___')
print(run1)
print('  ')

program2.measure_all()
run2 = qvm.run(program2,[0,1])
print('___Measurement on both qubits___')
print(run2)

_____The Wavefunction_____
(0.5+0j)|00> + (0.5+0j)|01> + (0.5+0j)|10> + (0.5+0j)|11>
  
___Measurement on qubit 0___
[[0]]
  
___Measurement on both qubits___
[[0, 1]]


In the code above, we create the same system twice (but they are not correlated in any way), and go about making measurements on the system in two different ways.  The first way is a single qubit measurement, while the second is a measurement on the complete system.

In Pyquil, we can store the results of measurements onto classical bits of data for viewing later, which is exactly what is done in both cases.  As an example, consider a system of 3 qubits, but we only re interested on the final state of the 2$^{nd}$ qubit:

In [91]:
p = Program(H(0),H(1),H(2))
target_qubit = 1
creg_index = 8

p.measure(target_qubit,creg_index)
qvm.run(p,[0,1,2])

[[0, 0, 0]]

This code will always return [[0,0,0]]. This is because we purposfully did something strange in where we choose to store the result of the measurement.  Specifically, we told the program to make a measurement on qubit 1, **and then store** this result in the 8$^{th}$ classical register spot.  Meanwhile, in the run( ) function, we asked to see only the info stored in the classical register spots 0 - 2 ([0,1,2]), Which by default starts out as all zeros.

To see that we really did store the result of the measurement somewhere odd, let's extend the range of values we want to see in our classical register.  Run the following code several times until you see a 1 in the 8$^{th}$ spot of the vector (there's a 50% chance each time):

In [130]:
p2 = Program(H(0),H(1),H(2))
target_qubit = 1
creg_index = 8

p2.measure(target_qubit,creg_index)
qvm.run(p2,[0,1,2,3,4,5,6,7,8,9,10])

[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]

Sure enough, there it is. A bizzare thing to ask, but the code does as intended. 

Now, suppose we want to be more interactive with our quantum circuits beyond just single measurements.  Pyquil allows us to do this in several ways, but we must first point out an important notation.  In standard dirac notation, it common to write the ordering of qubits from left to right, for example:

qubit 1 = |$0\rangle$  |  qubit 2 = |$1\rangle$  |  qubit 3 = |$1\rangle$ $\rightarrow$ |$011 \rangle$

However, pyquil stores qubit 0 as the **rightmost** state, with all qubits being written from right to left in ascending order:

**Pyquil:** qubit 1 = |$0\rangle$  |  qubit 2 = |$1\rangle$  |  qubit 3 = |$1\rangle$ $\rightarrow$ |$110 \rangle$

Thus, as a simple function for converting Pyquil's notation to a more common notation, we need only use the following function:

In [254]:
def SD(qubit,total):
    standard_notation = abs((total-1)-qubit)
    return standard_notation

total_qubits = 6
target_qubit = 2

print(SD(target_qubit,total_qubits))
    

3


The function above is completely optional, so long as Pyquil's notation does not confuse you.  But as an example of why it may be useful for starting out, the following code shows a situation which **may** be confusing otherwise:

In [255]:
p3 = Program(H(0),H(1),H(2))
target_qubit = 0
creg_index = 0

wf1 = qvm.wavefunction(p3)
print('___Wavefunction before the measurement___')
print(wf1)
print('  ')

p3.measure(target_qubit,creg_index)
measured = qvm.run(p3,[creg_index])
print('Measuered State of Qubit 0: ',measured)
print('  ')

wf2 = qvm.wavefunction(p3)
print('___Wavefunction after the measurement___')
print(wf2)
print('  ')

___Wavefunction before the measurement___
(0.3535533906+0j)|000> + (0.3535533906+0j)|001> + (0.3535533906+0j)|010> + (0.3535533906+0j)|011> + (0.3535533906+0j)|100> + (0.3535533906+0j)|101> + (0.3535533906+0j)|110> + (0.3535533906+0j)|111>
  
Measuered State of Qubit 0:  [[0]]
  
___Wavefunction after the measurement___
(0.5+0j)|000> + (0.5+0j)|010> + (0.5+0j)|100> + (0.5+0j)|110>
  


If you are unaware of Pyquil's notation, the result from this code may suggest something is going horribly wrong - we measured qubit 0, which should collapse it's wavefunction, but the resulting wavefunction appears to still show it in a superpostion.

However, if you know to read the qubits from right to left, there is no issue here.  The rightmost qubit **is** qubit 0, and does indeed collapse down to the measurement result.  But if we would like qubit 0 to be the leftmost number when we look at the wavefunction:

In [159]:
p3 = Program(H(0),H(1),H(2))
target_qubit = SD(0,3)
creg_index = 0

wf1 = qvm.wavefunction(p3)
print('___Wavefunction before the measurement___')
print(wf1)
print('  ')

p3.measure(target_qubit,creg_index)
measured = qvm.run(p3,[creg_index])
print('Measuered State of Qubit 0: ',measured)
print('  ')

wf2 = qvm.wavefunction(p3)
print('___Wavefunction after the measurement___')
print(wf2)
print('  ')

___Wavefunction before the measurement___
(0.3535533906+0j)|000> + (0.3535533906+0j)|001> + (0.3535533906+0j)|010> + (0.3535533906+0j)|011> + (0.3535533906+0j)|100> + (0.3535533906+0j)|101> + (0.3535533906+0j)|110> + (0.3535533906+0j)|111>
  
Measuered State of Qubit 0:  [[0]]
  
___Wavefunction after the measurement___
(0.5+0j)|000> + (0.5+0j)|001> + (0.5+0j)|010> + (0.5+0j)|011>
  


As a general rule of thumb, it is probably best to learn to adopt the notation that Pyquil insists on.  But for the examples in this lesson, we will use the SD() function as a way of seeing results that match what most people are more  accustomed to.  Now, let us return to the previous code example and discuss what happened.

In the example above we can see that we start with an equal superpostion of all 8 combitions for qubits 0,1, and 2.  But after we measure the target qubit (qubit 0), we collapse it down to only one value.  Thus, the resulting wavefunction afterwards is a superposition of the remaining four 4 states that all have the target qubit's measured value.

Later we shall see examples of entangled states, where measuring one qubit collapses the wavefunction of other qubits.  Here, qubits 1 and 2 are in seperable states from qubit 1, so even after the measurement they could still be in either |0$\rangle$ or |1$\rangle$.

Lastly, suppose we want to apply a gate to our qubits, but we don't want to specify it in Program().  This is a very common ask, since we can't always specify every single step of an algorithm from the beginning.  To do this, we use the function .inst(), which can be thought of as 'instantly' applying gate instructions to our program:

In [165]:
p4 = Program(H(0),H(1))
target_qubit = SD(1,2)
creg_index = 1

p4.measure(target_qubit,creg_index)
m4 = qvm.run(p4,[creg_index])
print('Measuered State of Qubit 1: ',m4)
print('  ')


print('__Wavefunction after measuring qubit 1___')
print(qvm.wavefunction(p4))
print('  ')

p4.inst(H(0))
print('__Wavefunction after .inst(H(0))___')
print(qvm.wavefunction(p4))

Measuered State of Qubit 1:  [[1]]
  
__Wavefunction after measuring qubit 1___
(0.7071067812+0j)|01> + (0.7071067812+0j)|11>
  
__Wavefunction after .inst(H(0))___
(0.5+0j)|00> + (-0.5+0j)|01> + (0.5+0j)|10> + (-0.5+0j)|11>


As we can see, our initial program is only two gates: Hadamards on qubits 0 and 1, followed my a measurement.  Then, using the .inst() function, we apply another gate after the measurement, Hadamard on qubit 0.

This concludes our brief explination of programs and measurements.  We will now proceed with working through each gate provided in the Pyquil libraray

-------
# Single Qubit Gates
-------

All of the following gates act on a single qubit.

## I

The single qubit Identity Gate

$$
\begin{bmatrix} 
1 & 0\\ 
0 & 1
\end{bmatrix}
$$

The effect of this gate renders the qubit's state unchanged.

In [256]:
from pyquil.gates import I

p_I = Program()
print('Initial State: ',qvm.wavefunction(p_I))
p_I.inst(I(0))
print('Identity: ',qvm.wavefunction(p_I))

Initial State:  (1+0j)|0>
Identity:  (1+0j)|0>


## Hadamard (H)

The single qubit Identity Gate

$$
\begin{bmatrix} 
1 & 1\\ 
1 & -1
\end{bmatrix}
$$

The effect of this gate is as follows:

$$\textbf{H} |0\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$$

$$\textbf{H} |1\rangle = \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$$

This gate results in a qubit being in a 50 / 50 superposition of states |$0\rangle$ and |$1\rangle$.

In [182]:
from pyquil.gates import H, X

p_H = Program(H(0))
print('H on |0>: ',qvm.wavefunction(p_H))
p_H = Program(X(0),H(0))
print('H on |1>: ',qvm.wavefunction(p_H))

H on |0>:  (0.7071067812+0j)|0> + (0.7071067812+0j)|1>
H on |1>:  (0.7071067812+0j)|0> + (-0.7071067812+0j)|1>


# Pauli Operators
------

## X

$$
\begin{bmatrix} 
0 & 1\\ 
1 & 0
\end{bmatrix}
$$

The effect of this gate flip's a qubits |0$\rangle$ and |1$\rangle$ amplitudes

In [174]:
from pyquil.gates import X
p_X = Program()
print('Initial qubit 0: ',qvm.wavefunction(p_X))
p_X.inst(X(0))
print('X on qubit 0: ',qvm.wavefunction(p_X))
p_X.inst(H(0))
print('H on qubit 0: ',qvm.wavefunction(p_X))
p_X.inst(X(0))
print('X on qubit 0: ',qvm.wavefunction(p_X))
p_X.inst(H(0))
print('H on qubit 0: ',qvm.wavefunction(p_X))

Initial qubit 0:  (1+0j)|0>
X on qubit 0:  (1+0j)|1>
H on qubit 0:  (0.7071067812+0j)|0> + (-0.7071067812+0j)|1>
X on qubit 0:  (-0.7071067812+0j)|0> + (0.7071067812+0j)|1>
H on qubit 0:  (-1+0j)|1>


## Y

$$
\begin{bmatrix} 
0 & -i\\ 
i & 0
\end{bmatrix}
$$

The effect of this gate flip's a qubits |0$\rangle$ and |1$\rangle$ amplitudes and multiplys by imaginary numbers (phase)

In [173]:
from pyquil.gates import Y
p_Y = Program()
print('Initial qubit 0: ',qvm.wavefunction(p_Y))
p_Y.inst(Y(0))
print('Y on qubit 0: ',qvm.wavefunction(p_Y))
p_Y.inst(H(0))
print('H on qubit 0: ',qvm.wavefunction(p_Y))
p_Y.inst(Y(0))
print('Y on qubit 0: ',qvm.wavefunction(p_Y))
p_Y.inst(H(0))
print('H on qubit 0: ',qvm.wavefunction(p_Y))

Initial qubit 0:  (1+0j)|0>
Y on qubit 0:  1j|1>
H on qubit 0:  0.7071067812j|0> + -0.7071067812j|1>
Y on qubit 0:  (-0.7071067812+0j)|0> + (-0.7071067812+0j)|1>
H on qubit 0:  (-1+0j)|0>


## Z

$$
\begin{bmatrix} 
1 & 0\\ 
0 & -1
\end{bmatrix}
$$

The effect of this gate leaves a qubit's |0$\rangle$ amplitude unchanged, while multiplying by -1 (phase) to a qubit's |1$\rangle$ amplitude.

In [177]:
from pyquil.gates import Z
p_Z = Program()
print('Initial qubit 0: ',qvm.wavefunction(p_Z))
p_Z.inst(Z(0))
print('Z on qubit 0: ',qvm.wavefunction(p_Z))
p_Z.inst(H(0))
print('H on qubit 0: ',qvm.wavefunction(p_Z))
p_Z.inst(Z(0))
print('Z on qubit 0: ',qvm.wavefunction(p_Z))
p_Z.inst(H(0))
print('H on qubit 0: ',qvm.wavefunction(p_Z))

Initial qubit 0:  (1+0j)|0>
Z on qubit 0:  (1+0j)|0>
H on qubit 0:  (0.7071067812+0j)|0> + (0.7071067812+0j)|1>
Z on qubit 0:  (0.7071067812+0j)|0> + (-0.7071067812+0j)|1>
H on qubit 0:  (1+0j)|1>


# Phase Gates
------

## PHASE  (R$_{\phi}$)

$$
\begin{bmatrix} 
1 & 0\\ 
0 & e^{i\phi}
\end{bmatrix}
$$

A gate similar to the Z gate.  It leaves a qubit's |0$\rangle$ amplitude unchanged, while multiplying by a phase $e^{i\phi}$ to a qubit's |1$\rangle$ amplitude.

In [191]:
import math as m
from pyquil.gates import PHASE
phi = m.pi
print('phi = pi -- this is equivilant to a Z gate')
print('  ')
p_Rp = Program(H(0))
print('Initial Mixed State: ',qvm.wavefunction(p_Rp))
p_Rp.inst(PHASE(phi,0))
print('S Gate: ',qvm.wavefunction(p_Rp))
p_Rp.inst(PHASE(phi,0))
print('S Gate: ',qvm.wavefunction(p_Rp))

phi = pi -- this is equivilant to a Z gate
  
Initial Mixed State:  (0.7071067812+0j)|0> + (0.7071067812+0j)|1>
S Gate:  (0.7071067812+0j)|0> + (-0.7071067812+0j)|1>
S Gate:  (0.7071067812+0j)|0> + (0.7071067812+0j)|1>


## S

A pre-defined gate for R$_{\phi}$, $\phi$=$\frac{\pi}{2}$

$$
\begin{bmatrix} 
1 & 0\\ 
0 & i
\end{bmatrix}
$$

A gate similar to the Z gate.  It leaves a qubit's |0$\rangle$ amplitude unchanged, while multiplying by i (phase) to a qubit's |1$\rangle$ amplitude.

In [187]:
from pyquil.gates import S
p_S = Program(H(0))
print('Initial Mixed State: ',qvm.wavefunction(p_S))
p_S.inst(S(0))
print('S Gate: ',qvm.wavefunction(p_S))
p_S.inst(S(0))
print('S Gate: ',qvm.wavefunction(p_S))

Initial Mixed State:  (0.7071067812+0j)|0> + (0.7071067812+0j)|1>
S Gate:  (0.7071067812+0j)|0> + 0.7071067812j|1>
S Gate:  (0.7071067812+0j)|0> + (-0.7071067812+0j)|1>


## T

A pre-defined gate for R$_{\phi}$, $\phi$=$\frac{\pi}{4}$

$$
\begin{bmatrix} 
1 & 0\\ 
0 & e^{i\frac{\pi}{4}}
\end{bmatrix}
$$

A gate similar to the Z gate.  It leaves a qubit's |0$\rangle$ amplitude unchanged, while multiplying by $e^{i\frac{\pi}{4}}$ (phase) to a qubit's |1$\rangle$ amplitude.

In [188]:
from pyquil.gates import T
p_T = Program(X(0))
print('Initial State: ',qvm.wavefunction(p_T))
p_T.inst(T(0))
print('T Gate: ',qvm.wavefunction(p_T))
p_T.inst(T(0))
print('T Gate: ',qvm.wavefunction(p_T))

Initial State:  (1+0j)|1>
S Gate:  (0.7071067812+0.7071067812j)|1>
S Gate:  1j|1>


# Rotation Gates
-------

The follow gates all represent rotations of a state on a Bloch spehere.  A Bloch sphere is a visual representation that maps the state of a qubit to a location on the surface of a sphere, radius = 1.  An image of a Bloch sphere and it's axes is given below:
![title](Bloch_Sphere.png)

## R$_x$($\theta$)

A rotation gate, representing a rotation around the x-axis on a Bloch Sphere

$$
\begin{bmatrix} 
cos(\frac{\theta}{2}) & -i \cdot sin(\frac{\theta}{2})\\ 
-i \cdot sin(\frac{\theta}{2}) & cos(\frac{\theta}{2})
\end{bmatrix}
$$


In [219]:
from pyquil.gates import RX
theta = m.pi/2
p_Rx = Program()
print('Initial State: ',qvm.wavefunction(p_Rx),'   = |0>')
print('theta = pi/2 -- rotations by 90 degrees around the x-axis (counter-clockwise)')
print('  ')
p_Rx.inst(RX(theta,0))
print('Rx(90) Gate: ',qvm.wavefunction(p_Rx),'   = -Y')
p_Rx.inst(RX(theta,0))
print('Rx(90) Gate: ',qvm.wavefunction(p_Rx),'   = |1>')
p_Rx.inst(RX(theta,0))
print('Rx(90) Gate: ',qvm.wavefunction(p_Rx),'   = Y')
p_Rx.inst(RX(theta,0))
print('Rx(90) Gate: ',qvm.wavefunction(p_Rx),'   = |0>')

Initial State:  (1+0j)|0>    = |0>
theta = pi/2 -- rotations by 90 degrees around the x-axis (counter-clockwise)
  
Rx(90) Gate:  (0.7071067812+0j)|0> + -0.7071067812j|1>    = -Y
Rx(90) Gate:  -1j|1>    = |1>
Rx(90) Gate:  (-0.7071067812+0j)|0> + -0.7071067812j|1>    = Y
Rx(90) Gate:  (-1+0j)|0>    = |0>


## R$_y$($\theta$)

A rotation gate, representing a rotation around the y-axis on a Bloch Sphere

$$
\begin{bmatrix} 
cos(\frac{\theta}{2}) & -sin(\frac{\theta}{2})\\ 
sin(\frac{\theta}{2}) & cos(\frac{\theta}{2})
\end{bmatrix}
$$


In [220]:
from pyquil.gates import RY
theta = m.pi/2
p_Ry = Program()
print('Initial State: ',qvm.wavefunction(p_Ry),'   = |0>')
print('theta = pi/2 -- rotations by 90 degrees around the y-axis (counter-clockwise)')
print('  ')
p_Ry.inst(RY(theta,0))
print('Ry(90) Gate: ',qvm.wavefunction(p_Ry),'   = X')
p_Ry.inst(RY(theta,0))
print('Ry(90) Gate: ',qvm.wavefunction(p_Ry),'   = |1>')
p_Ry.inst(RY(theta,0))
print('Ry(90) Gate: ',qvm.wavefunction(p_Ry),'   = -X')
p_Ry.inst(RY(theta,0))
print('Ry(90) Gate: ',qvm.wavefunction(p_Ry),'   = |0>')


Initial State:  (1+0j)|0>    = |0>
theta = pi/2 -- rotations by 90 degrees around the y-axis (counter-clockwise)
  
Ry(90) Gate:  (0.7071067812+0j)|0> + (0.7071067812+0j)|1>    = X
Ry(90) Gate:  (1+0j)|1>    = |1>
Ry(90) Gate:  (-0.7071067812+0j)|0> + (0.7071067812+0j)|1>    = -X
Ry(90) Gate:  (-1+0j)|0>    = |0>


## R$_z$($\theta$)

A rotation gate, representing a rotation around the z-axis on a Bloch Sphere

$$
\begin{bmatrix} 
e^{\frac{-i\theta}{2}} & 0\\ 
0 & e^{\frac{i\theta}{2}}
\end{bmatrix}
$$


In [228]:
from pyquil.gates import RZ
theta = m.pi/2
p_Rz = Program(H(0))
print('Initial State: ',qvm.wavefunction(p_Rz),' = X')
print('theta = pi -- rotations by 90 degrees around the z-axis (counter-clockwise)')
print('  ')
p_Rz.inst(RZ(theta,0))
print('Rz(90) Gate: ',qvm.wavefunction(p_Rz),'   = Y')
p_Rz.inst(RZ(theta,0))
print('Rz(90) Gate: ',qvm.wavefunction(p_Rz),'   = -X')
p_Rz.inst(RZ(theta,0))
print('Rz(90) Gate: ',qvm.wavefunction(p_Rz),'   = -Y')
p_Rz.inst(RZ(theta,0))
print('Rz(90) Gate: ',qvm.wavefunction(p_Rz),'   = X')

Initial State:  (0.7071067812+0j)|0> + (0.7071067812+0j)|1>  = X
theta = pi -- rotations by 90 degrees around the z-axis (counter-clockwise)
  
Rz(90) Gate:  (0.5-0.5j)|0> + (0.5+0.5j)|1>    = Y
Rz(90) Gate:  -0.7071067812j|0> + 0.7071067812j|1>    = -X
Rz(90) Gate:  (-0.5-0.5j)|0> + (-0.5+0.5j)|1>    = -Y
Rz(90) Gate:  (-0.7071067812+0j)|0> + (-0.7071067812+0j)|1>    = X


# Two Qubit Control Gates
---------

All of the following gates act on 2 qubits.

## CNOT

This gate uses a 'target qubit' and a 'control qubit'.  The effect of this gate is as follows:

$$\textbf{CNOT} \hspace{.2cm} |00\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |00\rangle $$
$$\textbf{CNOT} \hspace{.2cm} |01\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |01\rangle $$
$$\textbf{CNOT} \hspace{.2cm} |10\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |11\rangle $$
$$\textbf{CNOT} \hspace{.2cm} |11\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |10\rangle $$

In this notation, the first qubit is the control and the second qubit is the target.  If the first qubit is in the state |$0\rangle$, nothing happens to the second qubit.  If the first qubit is in the state |$1\rangle$, then the amplitudes of the second qubit are flipped.

Another way to think of this gate is as a 'control-X' gate, where the state of the control qubit determines whether or not an X gate is applied to the target qubit.

$$
\begin{bmatrix} 
1 & 0 & 0 & 0\\ 
0 & 1 & 0 & 0\\ 
0 & 0 & 0 & 1\\ 
0 & 0 & 1 & 0\\ 
\end{bmatrix}
$$


In [240]:
from pyquil.gates import CNOT
control_qubit = SD(0,2)
target_qubit  = SD(1,2)

p_CNOT = Program(I(control_qubit),I(target_qubit))
print('Initial State: ',qvm.wavefunction(p_CNOT))
p_CNOT.inst(CNOT(control_qubit,target_qubit))
print('After CNOT: ',qvm.wavefunction(p_CNOT))
print('  ')

p_CNOT = Program(X(control_qubit),I(target_qubit))
print('Initial State: ',qvm.wavefunction(p_CNOT))
p_CNOT.inst(CNOT(control_qubit,target_qubit))
print('After CNOT: ',qvm.wavefunction(p_CNOT))
print('  ')

p_CNOT = Program(I(control_qubit),X(target_qubit))
print('Initial State: ',qvm.wavefunction(p_CNOT))
p_CNOT.inst(CNOT(control_qubit,target_qubit))
print('After CNOT: ',qvm.wavefunction(p_CNOT))
print('  ')

p_CNOT = Program(X(control_qubit),X(target_qubit))
print('Initial State: ',qvm.wavefunction(p_CNOT))
p_CNOT.inst(CNOT(control_qubit,target_qubit))
print('After CNOT: ',qvm.wavefunction(p_CNOT))
print('  ')

Initial State:  (1+0j)|00>
After CNOT:  (1+0j)|00>
  
Initial State:  (1+0j)|10>
After CNOT:  (1+0j)|11>
  
Initial State:  (1+0j)|01>
After CNOT:  (1+0j)|01>
  
Initial State:  (1+0j)|11>
After CNOT:  (1+0j)|10>
  


The code above confirms the effect of the CNOT gate.  But this effect also applies to mixed states (which is where it's power comes from in more complex algorithms):

In [241]:
from pyquil.gates import CNOT
control_qubit = SD(0,2)
target_qubit  = SD(1,2)

p_CNOT = Program(X(control_qubit),H(control_qubit),I(target_qubit))
print('Initial State: ',qvm.wavefunction(p_CNOT))
p_CNOT.inst(CNOT(control_qubit,target_qubit))
print('After CNOT: ',qvm.wavefunction(p_CNOT))
print('  ')

Initial State:  (0.7071067812+0j)|00> + (-0.7071067812+0j)|10>
After CNOT:  (0.7071067812+0j)|00> + (-0.7071067812+0j)|11>
  


## CZ (Control-Z)

The control-Z gate works similarly to the CNOT gate, only instead of flipping the target qubit (applying an X gate), we apply a Z gate.

$$\textbf{CZ} \hspace{.2cm} |00\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |00\rangle $$
$$\textbf{CZ} \hspace{.2cm} |01\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |01\rangle $$
$$\textbf{CZ} \hspace{.2cm} |10\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |10\rangle $$
$$\textbf{CZ} \hspace{.2cm} |11\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} -|11\rangle $$

Recall that a Z gates leaves a qubit in the state |$0\rangle$ untouched, while flipping the sign on a qubit in the state |$1\rangle$.  Thus, the CZ gate only affects the state |$11\rangle$, as shown above.

$$
\begin{bmatrix} 
1 & 0 & 0 & 0\\ 
0 & 1 & 0 & 0\\ 
0 & 0 & 1 & 0\\ 
0 & 0 & 0 & -1\\ 
\end{bmatrix}
$$

In [242]:
from pyquil.gates import CZ
control_qubit = SD(0,2)
target_qubit  = SD(1,2)

p_CZ = Program(H(control_qubit),H(target_qubit))
print('Initial State: ',qvm.wavefunction(p_CZ))
p_CZ.inst(CZ(control_qubit,target_qubit))
print('After CZ: ',qvm.wavefunction(p_CZ))


Initial State:  (0.5+0j)|00> + (0.5+0j)|01> + (0.5+0j)|10> + (0.5+0j)|11>
After CZ:  (0.5+0j)|00> + (0.5+0j)|01> + (0.5+0j)|10> + (-0.5+0j)|11>


## Control Phase Gates

The following four gates will be bundled together, since they all represent the same general effect

#### CPHASE($\phi$) |$11\rangle$

$$
\begin{bmatrix} 
1 & 0 & 0 & 0\\ 
0 & 1 & 0 & 0\\ 
0 & 0 & 1 & 0\\ 
0 & 0 & 0 & e^{i\phi}\\ 
\end{bmatrix}
$$

#### CPHASE($\phi$) |$10\rangle$

$$
\begin{bmatrix} 
1 & 0 & 0 & 0\\ 
0 & 1 & 0 & 0\\ 
0 & 0 & e^{i\phi} & 0\\ 
0 & 0 & 0 & 1\\ 
\end{bmatrix}
$$

#### CPHASE($\phi$) |$01\rangle$

$$
\begin{bmatrix} 
1 & 0 & 0 & 0\\ 
0 & e^{i\phi} & 0 & 0\\ 
0 & 0 & 1 & 0\\ 
0 & 0 & 0 & 1\\ 
\end{bmatrix}
$$

#### CPHASE($\phi$) |$00\rangle$

$$
\begin{bmatrix} 
e^{i\phi} & 0 & 0 & 0\\ 
0 & 1 & 0 & 0\\ 
0 & 0 & 1 & 0\\ 
0 & 0 & 0 & 1\\ 
\end{bmatrix}
$$


By standard convention the CPHASE gate is the one that acts on the state |$11\rangle$, but all of the gates are the same in principle.  Each of the variations of the CPHASE gate picks out one of the four states and applies a phase operation.

In [253]:
from pyquil.gates import CPHASE,CPHASE10,CPHASE01,CPHASE00
phi = m.pi
control_qubit = SD(0,2)
target_qubit  = SD(1,2)

p_CP11 = Program(H(control_qubit),H(target_qubit))
p_CP10 = Program(H(control_qubit),H(target_qubit))
p_CP01 = Program(H(control_qubit),H(target_qubit))
p_CP00 = Program(H(control_qubit),H(target_qubit))
print('phi = pi    --   same effect as a Z gate  ')
print('_____Initial State:_____')
print(qvm.wavefunction(p_CP11))
print(' ')

p_CP11.inst(CPHASE(phi,control_qubit,target_qubit))
print('CPHASE: ',qvm.wavefunction(p_CP11))
p_CP10.inst(CPHASE10(phi,control_qubit,target_qubit))
print('CPHASE10: ',qvm.wavefunction(p_CP10))
p_CP01.inst(CPHASE01(phi,control_qubit,target_qubit))
print('CPHASE01: ',qvm.wavefunction(p_CP01))
p_CP00.inst(CPHASE00(phi,control_qubit,target_qubit))
print('CPHASE00: ',qvm.wavefunction(p_CP00))


phi = pi    --   same effect as a Z gate  
_____Initial State:_____
(0.5+0j)|00> + (0.5+0j)|01> + (0.5+0j)|10> + (0.5+0j)|11>
 
CPHASE:  (0.5+0j)|00> + (0.5+0j)|01> + (0.5+0j)|10> + (-0.5+0j)|11>
CPHASE10:  (0.5+0j)|00> + (0.5+0j)|01> + (-0.5+0j)|10> + (0.5+0j)|11>
CPHASE01:  (0.5+0j)|00> + (-0.5+0j)|01> + (0.5+0j)|10> + (0.5+0j)|11>
CPHASE00:  (-0.5+0j)|00> + (0.5+0j)|01> + (0.5+0j)|10> + (0.5+0j)|11>


# Swapping Gates
------

The following four gates all perform a "swap", whereby the states of two qubits are

## SWAP

The SWAP gate causes two qubits to trade states.

$$\textbf{SWAP} \hspace{.2cm} |00\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |00\rangle $$
$$\textbf{SWAP} \hspace{.2cm} |01\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |11\rangle $$
$$\textbf{SWAP} \hspace{.2cm} |10\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |01\rangle $$
$$\textbf{SWAP} \hspace{.2cm} |11\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |11\rangle $$

A simple way of viewing the effect of this gate is it switches the numbers on each state.  As a result, we can see that the SWAP gate has no effect on the states |$00\rangle$ and |$11\rangle$.

$$
\begin{bmatrix} 
1 & 0 & 0 & 0\\ 
0 & 0 & 1 & 0\\ 
0 & 1 & 0 & 0\\ 
0 & 0 & 0 & 1\\ 
\end{bmatrix}
$$

In [270]:
from pyquil.gates import SWAP
qubit0 = SD(0,2)
qubit1 = SD(1,2)

p_SWAP = Program(H(0),H(1),CPHASE10(m.pi,qubit0,qubit1))
print('Initial State: ',qvm.wavefunction(p_SWAP))
print(' ')

p_SWAP.inst(SWAP(qubit0,qubit1))
print('  SWAP: ',qvm.wavefunction(p_SWAP))


Initial State:  (0.5+0j)|00> + (0.5+0j)|01> + (-0.5+0j)|10> + (0.5+0j)|11>
 
  SWAP:  (0.5+0j)|00> + (-0.5+0j)|01> + (0.5+0j)|10> + (0.5+0j)|11>


## ISWAP

This gate produces a similar effect as the SWAP gate, but applies a a phase operation of $\textit{i}$ to the swapped states.

$$
\begin{bmatrix} 
1 & 0 & 0 & 0\\ 
0 & 0 & i & 0\\ 
0 & i & 0 & 0\\ 
0 & 0 & 0 & 1\\ 
\end{bmatrix}
$$

In [271]:
from pyquil.gates import ISWAP
qubit0 = SD(0,2)
qubit1 = SD(1,2)

p_ISWAP = Program(H(0),H(1),CPHASE10(m.pi,qubit0,qubit1))
print('Initial State: ',qvm.wavefunction(p_ISWAP))
print(' ')

p_ISWAP.inst(ISWAP(qubit0,qubit1))
print('  ISWAP: ',qvm.wavefunction(p_ISWAP))

Initial State:  (0.5+0j)|00> + (0.5+0j)|01> + (-0.5+0j)|10> + (0.5+0j)|11>
 
  ISWAP:  (0.5+0j)|00> + (-0-0.5j)|01> + 0.5j|10> + (0.5+0j)|11>


## ISWAP($\phi$)

A more general version of the ISWAP gate, whereby one can specify any phase operation to be applied to the swapped states.

$$
\begin{bmatrix} 
1 & 0 & 0 & 0\\ 
0 & 0 & e^{i\phi} & 0\\ 
0 & e^{i\phi} & 0 & 0\\ 
0 & 0 & 0 & 1\\ 
\end{bmatrix}
$$

In [272]:
from pyquil.gates import PSWAP
qubit0 = SD(0,2)
qubit1 = SD(1,2)
phi = m.pi
print('phi = pi   --   same effect as a -1 phase operation')

p_PSWAP = Program(H(0),H(1),CPHASE10(m.pi,qubit0,qubit1))
print('Initial State: ',qvm.wavefunction(p_PSWAP))
print(' ')

p_PSWAP.inst(PSWAP(phi,qubit0,qubit1))
print('  PSWAP(pi): ',qvm.wavefunction(p_PSWAP))

phi = pi   --   same effect as a -1 phase operation
Initial State:  (0.5+0j)|00> + (0.5+0j)|01> + (-0.5+0j)|10> + (0.5+0j)|11>
 
  PSWAP(pi):  (0.5+0j)|00> + (0.5+0j)|01> + (-0.5+0j)|10> + (0.5+0j)|11>


# 3 Qubit Control Gates
----

The following two gates take 3 qubites as inputs.  They are essentially higher order versions of the CNOT and SWAP gates, adding one extra control qubit to each.

## CSWAP

The control-swap gate uses a control qubit tp determine whether or not to apply a SWAP gate to two target qubits.  If the control qubit is in the state |$1\rangle$, then a SWAP gate is performed.  Examples:

$$\textbf{CSWAP} \hspace{.2cm} |010\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |010\rangle $$
$$\textbf{CSWAP} \hspace{.2cm} |101\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |110\rangle $$
$$\textbf{CSWAP} \hspace{.2cm} |110\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |101\rangle $$
$$\textbf{CSWAP} \hspace{.2cm} |111\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |111\rangle $$

This gate is also sometimes refered to as a Fredkin Gate.

$$
\begin{bmatrix} 
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\ 
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\ 
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\ 
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\ 
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\\ 
\end{bmatrix}
$$


In [279]:
from pyquil.gates import CSWAP
control_qubit = SD(0,3)
qubit1 = SD(1,3)
qubit2 = SD(2,3)

p_CSWAP = Program(H(0),H(1),H(2),Z(0),H(0))
print('Initial State: ',qvm.wavefunction(p_CSWAP))
print(' ')

p_CSWAP.inst(CSWAP(control_qubit,qubit1,qubit2))
print('  CSWAP: ',qvm.wavefunction(p_CSWAP))

Initial State:  (0.5+0j)|001> + (0.5+0j)|011> + (0.5+0j)|101> + (0.5+0j)|111>
 
  CSWAP:  (0.5+0j)|001> + (0.5+0j)|011> + (0.5+0j)|110> + (0.5+0j)|111>


## CCNOT

The control-control not gate uses two control qubits to determine if an X gate is applied to a single target qubit.  In principle, one can have as many control gates, not just up to two. Examples:

$$\textbf{CCNOT} \hspace{.2cm} |010\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |010\rangle $$
$$\textbf{CCNOT} \hspace{.2cm} |101\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |101\rangle $$
$$\textbf{CCNOT} \hspace{.2cm} |110\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |111\rangle $$
$$\textbf{CCNOT} \hspace{.2cm} |111\rangle \hspace{.25cm} \rightarrow \hspace{.25cm} |110\rangle $$

This gate is also sometimes refered to as a Toffoli Gate.

$$
\begin{bmatrix} 
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\ 
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\ 
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\ 
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\\ 
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\ 
\end{bmatrix}
$$

In [282]:
from pyquil.gates import CCNOT
control_qubit0 = SD(0,3)
control_qubit1 = SD(1,3)
qubit2 = SD(2,3)

p_CCNOT = Program(H(0),H(1),H(2),CPHASE10(m.pi,control_qubit1,qubit2),CPHASE10(m.pi,control_qubit0,control_qubit1))
print('Initial State: ',qvm.wavefunction(p_CCNOT))
print(' ')

p_CCNOT.inst(CCNOT(control_qubit0,control_qubit1,qubit2))
print('  CCNOT: ',qvm.wavefunction(p_CCNOT))

Initial State:  (0.3535533906+0j)|000> + (0.3535533906+0j)|001> + (-0.3535533906+0j)|010> + (0.3535533906+0j)|011> + (-0.3535533906+0j)|100> + (-0.3535533906+0j)|101> + (-0.3535533906+0j)|110> + (0.3535533906+0j)|111>
 
  CCNOT:  (0.3535533906+0j)|000> + (0.3535533906+0j)|001> + (-0.3535533906+0j)|010> + (0.3535533906+0j)|011> + (-0.3535533906+0j)|100> + (-0.3535533906+0j)|101> + (0.3535533906+0j)|110> + (-0.3535533906+0j)|111>


-----
This concludes all of the quantum gates provided by the pquil.gates file.  However, Pyquil allows the user to create more gates beyond this basic set, so long as one can specifiy the gate's matrix representation.